# Домашнее задание 4

Максимальный балл за домашнюю работу 10.
Необходимо выполнять все задания в ноутбуке с помощью чистого SQL и SQLAlchemy ORM:
- осуществить подключение к БД
- каждое задание оформить в отдельной ячейке
- результат выполнения запросов записать в pandas dataframe

In [36]:
import psycopg2
import pandas as pd

In [107]:
connect = psycopg2.connect(
    host = "ep-cool-dream-a299xjx0.eu-central-1.aws.neon.tech",
    port = 5432,
    database = "club",
    user = "Tokimaro",
    password = "EA6yiwMkqIW7",
    options="-c search_path=cd"
)

In [108]:
cursor = connect.cursor()

In [109]:
from sqlalchemy import create_engine


In [110]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase demo"""
    login = 'Tokimaro'
    password = 'EA6yiwMkqIW7'
    host = 'ep-cool-dream-a299xjx0.eu-central-1.aws.neon.tech'
    port = '5432'
    db_name = 'club'
    dbschema='cd' # Searches left-to-right
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo ,  connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [111]:
engine = connection_yandex_cloud_demo(echo=False)

In [112]:
from sqlalchemy.schema import MetaData

metadata_demo_db = MetaData()
metadata_demo_db.reflect(bind=engine);

In [113]:
from sqlalchemy import Table
from sqlalchemy import select
facilities = Table('facilities', metadata_demo_db, autoload=True)
bookings = Table('bookings', metadata_demo_db, autoload=True)
members = Table('members', metadata_demo_db, autoload=True)

## Задание 1 (1 балл)

Сформируйте упорядоченный список из первых 10 фамилий в таблице участников?  
Список не должен содержать дубликатов.

In [92]:
sql = '''
    SELECT DISTINCT surname 
    FROM members 
    ORDER BY surname ASC 
    LIMIT 10
'''
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['surname']
df

,surname
0,Bader
1,Baker
2,Boothe
3,Butters
4,Coplin
5,Crumpet
6,Dare
7,Farrell
8,GUEST
9,Genting


In [37]:
sql_query = select([members.c.surname.distinct()]).\
            order_by(members.c.surname).\
            limit(10)
df = pd.DataFrame(engine.execute(sql_query).fetchall())
df.columns = ['surname']
df

,surname
0,Bader
1,Baker
2,Boothe
3,Butters
4,Coplin
5,Crumpet
6,Dare
7,Farrell
8,GUEST
9,Genting


## Задание 2  (1 балл)

Выведите имя и фамилию последнего зарегистрировавшегося участника, а также дату.

In [93]:
sql = '''
    SELECT firstname, surname, joindate 
    FROM members 
    ORDER BY joindate DESC 
    LIMIT 1
'''
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['firstname', 'surname', 'joindate']
df

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45


In [39]:
from sqlalchemy import desc

sql_query = select([members.c.firstname,members.c.surname,members.c.joindate ]).\
            order_by(desc(members.c.joindate)).\
            limit(10)
df = pd.DataFrame(engine.execute(sql_query).fetchall())
df.columns = ['firstname', 'surname', 'joindate']
df

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45


## Задание 3 (2 балла)

Составьте список времени начала бронирования теннисных кортов на дату "2012-09-21".  
В итоговой выборке верните список пар времени начала и названий объектов, упорядоченных по времени.

In [94]:
sql = '''
    SELECT b.starttime::time ,f.name
    FROM bookings as b
    LEFT JOIN facilities as f
        ON f.facid = b.facid
    WHERE
        b.starttime::DATE = '2012-09-21'
    ORDER BY b.starttime
'''
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['stattime', 'name']
df

,stattime,name
0,08:00:00,Tennis Court 2
1,08:00:00,Tennis Court 1
2,08:30:00,Massage Room 1
3,08:30:00,Snooker Table
4,08:30:00,Pool Table
...,...,...
64,19:00:00,Snooker Table
65,19:00:00,Pool Table
66,19:30:00,Massage Room 1
67,19:30:00,Pool Table


In [54]:
from sqlalchemy import func
sql_query = select([bookings.c.starttime.cast(Time),facilities.c.name]).\
            order_by(bookings.c.starttime).\
            select_from(bookings.\
            join(facilities)
            ).\
            where(bookings.c.starttime.cast(Date) == '2012-09-21')
df = pd.DataFrame(engine.execute(sql_query).fetchall())
df.columns = ['stattime', 'name']
df

,stattime,name
0,08:00:00,Tennis Court 2
1,08:00:00,Tennis Court 1
2,08:30:00,Massage Room 1
3,08:30:00,Snooker Table
4,08:30:00,Pool Table
...,...,...
64,19:00:00,Snooker Table
65,19:00:00,Pool Table
66,19:30:00,Massage Room 1
67,19:30:00,Pool Table


## Задание 4 (2 балла)

 Выведите список всех участников, включая человека, который их рекомендовал (если таковой имеется).  
 Убедитесь, что результаты упорядочены по (фамилия, имя).

In [95]:
sql = '''
SELECT m1.surname, m1.firstname,m1.address, m1.zipcode, m1.telephone, m1.joindate, COALESCE(m2.surname, '-') as reccomendedby_surname, COALESCE(m2.firstname, '-') as reccomendedby_firstname 
FROM members as m1
LEFT JOIN members as m2
ON m2.memid = m1.recommendedby
ORDER BY m1.surname, m1.firstname ASC
'''
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['surname', 'firstname' ,'address', 'zipcode', 'telephone', 'joindate' , 'reccomendedby_surname', 'reccomendedby_firstname']
df

,surname,firstname,address,zipcode,telephone,joindate,reccomendedby_surname,reccomendedby_firstname
0,Bader,Florence,"264 Ursula Drive, Westford",84923,(833) 499-3527,2012-08-10 17:52:03,Stibbons,Ponder
1,Baker,Anne,"55 Powdery Street, Boston",80743,844-076-5141,2012-08-10 14:23:22,Stibbons,Ponder
2,Baker,Timothy,"329 James Street, Reading",58393,833-941-0824,2012-08-15 10:34:25,Farrell,Jemima
3,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,2012-07-25 16:02:35,Rownam,Tim
4,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,2012-07-09 10:44:09,Smith,Darren
5,Coplin,Joan,"85 Bard Street, Bloomington, Boston",43533,(822) 499-2232,2012-08-29 08:32:41,Baker,Timothy
6,Crumpet,Erica,"Crimson Road, North Reading",75655,(811) 732-4816,2012-09-22 08:36:38,Smith,Tracy
7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,2012-07-25 08:59:12,Joplette,Janice
8,Farrell,David,"437 Granite Farm Road, Westford",43532,(855) 755-9876,2012-09-15 08:22:05,-,-
9,Farrell,Jemima,"103 Firth Avenue, North Reading",57392,(855) 016-0163,2012-08-10 14:28:01,-,-


In [73]:
from sqlalchemy.orm import aliased
from sqlalchemy.sql.functions import coalesce
m2 = aliased(members)
sql_query = select([members.c.surname, members.c.firstname,members.c.address, members.c.zipcode, members.c.telephone, members.c.joindate, coalesce(m2.c.surname, '-'), coalesce(m2.c.firstname, '-')]).\
join(m2, m2.c.memid == members.c.recommendedby, isouter = True).\
order_by(members.c.surname, members.c.firstname)
df = pd.DataFrame(engine.execute(sql_query).fetchall())
df.columns = ['surname', 'firstname' ,'address', 'zipcode', 'telephone', 'joindate' , 'reccomendedby_surname', 'reccomendedby_firstname']
df

,surname,firstname,address,zipcode,telephone,joindate,reccomendedby_surname,reccomendedby_firstname
0,Bader,Florence,"264 Ursula Drive, Westford",84923,(833) 499-3527,2012-08-10 17:52:03,Stibbons,Ponder
1,Baker,Anne,"55 Powdery Street, Boston",80743,844-076-5141,2012-08-10 14:23:22,Stibbons,Ponder
2,Baker,Timothy,"329 James Street, Reading",58393,833-941-0824,2012-08-15 10:34:25,Farrell,Jemima
3,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,2012-07-25 16:02:35,Rownam,Tim
4,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,2012-07-09 10:44:09,Smith,Darren
5,Coplin,Joan,"85 Bard Street, Bloomington, Boston",43533,(822) 499-2232,2012-08-29 08:32:41,Baker,Timothy
6,Crumpet,Erica,"Crimson Road, North Reading",75655,(811) 732-4816,2012-09-22 08:36:38,Smith,Tracy
7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,2012-07-25 08:59:12,Joplette,Janice
8,Farrell,David,"437 Granite Farm Road, Westford",43532,(855) 755-9876,2012-09-15 08:22:05,-,-
9,Farrell,Jemima,"103 Firth Avenue, North Reading",57392,(855) 016-0163,2012-08-10 14:28:01,-,-


## Задание 5 (2 балла)

Составьте список чила слотов забронированных по каждой услуге в сентябре 2012ого года.  

В выводе отобразите две колонки: `facid`, `Total Slots`, и отсортируйте по числу слотов.

In [96]:
sql = '''
SELECT facid, sum(slots) as "Total Slots" 
FROM bookings
WHERE date_part('month', starttime) = 9 AND date_part('year', starttime) = 2012
GROUP BY facid 
ORDER BY "Total Slots"
'''
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['facid', 'Total Slots']
df

,facid,Total Slots
0,5,122
1,3,422
2,7,426
3,8,471
4,6,540
5,2,570
6,1,588
7,0,591
8,4,648


In [77]:
from sqlalchemy.sql import func
sql_query = select([bookings.c.facid, func.sum(bookings.c.slots).label('Total Slots')]).\
            where((func.date_part('month', bookings.c.starttime) == 9) & (func.date_part('year', bookings.c.starttime) == 2012)).\
            group_by(bookings.c.facid).\
            order_by('Total Slots')
df = pd.DataFrame(engine.execute(sql_query).fetchall())
df.columns = ['facid', 'Total Slots']
df

,facid,Total Slots
0,5,122
1,3,422
2,7,426
3,8,471
4,6,540
5,2,570
6,1,588
7,0,591
8,4,648


## Задание 6 (2 балла)

Составьте список бронирований на день 2012-09-14, которые обойдутся участнику (или гостю) более чем в 30 долларов. Помните, что у гостя всегда идентификатор 0 и расходы гостей отличаются от расходов участников (расходы указаны за получасовой слот).

В выводе отобразите три колонки: `member`, `facility`, `cost` и отсортируйте по убыванию стоимости. В колонке `member` объединим имя и фамилию следующим образом:

`mems.firstname || ' ' || mems.surname as member`  
(в данном случае `mems` — это алиас на колонку `members`)

In [114]:
sql = '''
select m.firstname || ' ' || m.surname as member, f.name, a.price as cost 
from bookings as b
left join members m 
on m.memid = b.memid 
left join facilities f 
on f.facid = b.facid 
CROSS JOIN LATERAL
(select b.slots * f.membercost) a(Price)
where starttime::DATE = '2012-09-14' and b.memid != 0 and a.price > 30

union

select m.firstname || ' ' || m.surname as member, f.name, a.price as cost from bookings as b
left join members m on
m.memid = b.facid 
left join facilities f on
f.facid = b.facid CROSS JOIN LATERAL
(select b.slots * f.guestcost) a(Price)
where starttime::DATE = '2012-09-14' and b.memid = 0 and a.price > 30
order by cost desc
'''
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['member', 'facility', 'cost']
df

,member,facility,cost
0,Gerald Butters,Massage Room 2,320
1,Janice Joplette,Massage Room 1,160
2,Darren Smith,Tennis Court 2,150
3,Jemima Farrell,Massage Room 1,140
4,Darren Smith,Tennis Court 2,75
5,GUEST GUEST,Tennis Court 1,75
6,Jack Smith,Massage Room 1,70
7,Burton Tracy,Squash Court,70.0
8,Burton Tracy,Massage Room 1,70
9,Matthew Genting,Massage Room 1,70


In [130]:
sql_query = select([func.concat(members.c.firstname,' ',members.c.surname),facilities.c.name, (bookings.c.slots * facilities.c.membercost).label('price')]).\
            join(members, members.c.memid == bookings.c.memid, isouter = True).\
            join(facilities, facilities.c.facid == bookings.c.facid, isouter = True).\
            where(((bookings.c.slots * facilities.c.membercost) > 30) & (bookings.c.memid != 0) & (bookings.c.starttime.cast(Date)=='2012-09-14')) 
sql_query1 = select([func.concat(members.c.firstname,' ',members.c.surname),facilities.c.name, (bookings.c.slots * facilities.c.guestcost).label('price')]).\
            join(members, members.c.memid == bookings.c.memid, isouter = True).\
            join(facilities, facilities.c.facid == bookings.c.facid, isouter = True).\
            where(((bookings.c.slots * facilities.c.guestcost) > 30) & (bookings.c.memid == 0) & (bookings.c.starttime.cast(Date)=='2012-09-14'))
sql_query2 = sql_query.union(sql_query1).order_by(desc('price'))
df = pd.DataFrame(engine.execute(sql_query2).fetchall())
df.columns = ['member', 'facility', 'cost']
df

,member,facility,cost
0,GUEST GUEST,Massage Room 2,320
1,GUEST GUEST,Massage Room 1,160
2,GUEST GUEST,Tennis Court 2,150
3,Jemima Farrell,Massage Room 1,140
4,GUEST GUEST,Tennis Court 2,75
5,GUEST GUEST,Tennis Court 1,75
6,Jack Smith,Massage Room 1,70
7,GUEST GUEST,Squash Court,70.0
8,Burton Tracy,Massage Room 1,70
9,Matthew Genting,Massage Room 1,70
